# PyTau Verbose Functionality

This notebook demonstrates the verbose functionality in PyTau that provides detailed progress information during model initialization, fitting, and prediction.

## Key Features
- **Global verbose setting**: Control verbosity for all PyTau operations
- **Instance-specific verbose**: Override global setting for individual models
- **Context manager**: Temporarily change verbose setting
- **Detailed progress tracking**: See what's happening during model operations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add pytau to path
sys.path.append('../../')

from pytau import (
    ChangepointDetector, 
    fit_changepoint_model,
    set_verbose, 
    get_verbose, 
    verbose_context,
    reset_config
)

## Global Verbose Settings

PyTau provides global verbose control that affects all operations.

In [ ]:
# Check default verbose setting
print(f"Default verbose setting: {get_verbose()}")

# Enable verbose output globally
set_verbose(True)
print(f"After set_verbose(True): {get_verbose()}")

# Disable verbose output
set_verbose(False)
print(f"After set_verbose(False): {get_verbose()}")

# Verbose setting accepts various formats
set_verbose('on')   # String values
print(f"After set_verbose('on'): {get_verbose()}")

set_verbose(0)      # Integer values (0=False, non-zero=True)
print(f"After set_verbose(0): {get_verbose()}")

## Generate Test Data

In [ ]:
# Generate synthetic spike data for testing
np.random.seed(42)
n_trials = 15
n_neurons = 5
n_timepoints = 60
true_changepoint = 30

spike_data = np.zeros((n_trials, n_neurons, n_timepoints))

for trial in range(n_trials):
    for neuron in range(n_neurons):
        # State 1: Low firing rate
        spike_data[trial, neuron, :true_changepoint] = np.random.poisson(
            1.5, size=true_changepoint
        )
        # State 2: High firing rate
        spike_data[trial, neuron, true_changepoint:] = np.random.poisson(
            4.0, size=n_timepoints - true_changepoint
        )

print(f"Generated spike data shape: {spike_data.shape}")
print(f"True changepoint: {true_changepoint}")

## Demonstration 1: Global Verbose ON

In [ ]:
# Enable global verbose to see detailed output
set_verbose(True)
print("=== Global Verbose ON ===")

# Create and fit model - you'll see detailed progress
detector1 = ChangepointDetector(
    model_type='single_taste_poisson',
    n_states=2,
    n_iterations=100,  # Small number for demo
    random_state=42
)

print("\n--- Fitting Model ---")
detector1.fit(spike_data)

print("\n--- Making Predictions ---")
predictions1 = detector1.predict()

if 'changepoints' in predictions1:
    estimated_cp = predictions1['changepoints'][0]
    print(f"\nEstimated changepoint: {estimated_cp:.1f}")
    print(f"True changepoint: {true_changepoint}")
    print(f"Error: {abs(estimated_cp - true_changepoint):.1f}")

## Demonstration 2: Global Verbose OFF, Instance Verbose ON

In [ ]:
# Disable global verbose
set_verbose(False)
print("=== Global Verbose OFF, Instance Verbose ON ===")

# Create model with instance-specific verbose=True
detector2 = ChangepointDetector(
    model_type='single_taste_poisson',
    n_states=2,
    n_iterations=100,
    random_state=42,
    verbose=True  # Override global setting
)

print("\n--- Fitting Model (should show verbose output) ---")
detector2.fit(spike_data)

predictions2 = detector2.predict()

## Demonstration 3: Global Verbose ON, Instance Verbose OFF

In [ ]:
# Enable global verbose
set_verbose(True)
print("=== Global Verbose ON, Instance Verbose OFF ===")

# Create model with instance-specific verbose=False
detector3 = ChangepointDetector(
    model_type='single_taste_poisson',
    n_states=2,
    n_iterations=100,
    random_state=42,
    verbose=False  # Override global setting
)

print("\n--- Fitting Model (should be quiet) ---")
detector3.fit(spike_data)
print("Model fitted silently!")

predictions3 = detector3.predict()

## Demonstration 4: Context Manager

In [ ]:
# Set global verbose to False
set_verbose(False)
print("=== Context Manager Demo ===")
print(f"Global verbose before context: {get_verbose()}")

# Use context manager to temporarily enable verbose
with verbose_context(True):
    print(f"Verbose inside context: {get_verbose()}")
    
    # Operations inside context will be verbose
    detector4 = ChangepointDetector(
        model_type='single_taste_poisson',
        n_states=2,
        n_iterations=50,
        random_state=42
    )

print(f"Global verbose after context: {get_verbose()}")
print("Context manager restored original setting!")

## Demonstration 5: Convenience Function with Verbose

In [ ]:
# Test convenience function with verbose parameter
set_verbose(False)  # Global off
print("=== Convenience Function with Verbose ===")

# Use convenience function with verbose=True
detector5 = fit_changepoint_model(
    spike_data,
    model_type='single_taste_poisson',
    n_states=2,
    n_iterations=50,
    verbose=True,  # Enable verbose for this operation
    random_state=42
)

predictions5 = detector5.predict()
print("\nConvenience function completed!")

## Parameter Management with Verbose

In [ ]:
# Demonstrate parameter management with verbose
set_verbose(True)
print("=== Parameter Management ===")

detector = ChangepointDetector(verbose=False)

# Get parameters (includes verbose)
params = detector.get_params()
print(f"\nCurrent parameters:")
for key, value in params.items():
    print(f"  {key}: {value}")

# Update parameters including verbose
print("\n--- Updating Parameters ---")
detector.set_params(verbose=True, n_states=3, n_iterations=200)

updated_params = detector.get_params()
print(f"\nUpdated parameters:")
for key, value in updated_params.items():
    print(f"  {key}: {value}")

## Environment Variable Control

You can also control the default verbose setting using the `PYTAU_VERBOSE` environment variable:

```bash
# Enable verbose by default
export PYTAU_VERBOSE=true

# Disable verbose by default  
export PYTAU_VERBOSE=false
```

In [ ]:
# Reset configuration to defaults
print("=== Resetting Configuration ===")
print(f"Before reset: {get_verbose()}")

reset_config()
print(f"After reset: {get_verbose()}")
print("Configuration reset to defaults!")

## Summary

PyTau's verbose functionality provides:

1. **Global Control**: `set_verbose()` and `get_verbose()` for system-wide settings
2. **Instance Control**: `verbose` parameter in `ChangepointDetector` to override global settings
3. **Context Manager**: `verbose_context()` for temporary verbose changes
4. **Convenience Functions**: `verbose` parameter in `fit_changepoint_model()`
5. **Environment Variables**: `PYTAU_VERBOSE` for default settings
6. **Configuration Reset**: `reset_config()` to restore defaults

This gives you fine-grained control over when and where you want to see detailed progress information during your changepoint detection workflows.